In [ ]:
from math import sqrt
import numpy as np
from pyscf import gto, scf, cc, fci, dft
import matplotlib.pyplot as plt
from nbed.driver import NbedDriver

In [ ]:
def find_distance(r1, r2):
    return sqrt((r2[0]-r1[0])**2 + (r2[1]-r1[1])**2 + (r2[2]-r1[2])**2)

def find_point(r1, r2, d):
    R = find_distance(r1, r2)
    r2p = [r1[0] + (r2[0]-r1[0])*d/R, r1[1] + (r2[1]-r1[1])*d/R, r1[2] + (r2[2]-r1[2])*d/R]
    r2m = [r1[0] - (r2[0]-r1[0])*d/R, r1[1] - (r2[1]-r1[1])*d/R, r1[2] - (r2[2]-r1[2])*d/R]
    return r2p

def edit_xyz(filename, atom1_line_num, atom2_line_num, length, precision):
    with open(filename, 'r') as file:
        readlines = file.readlines()

    # ignoring white space so ['atom', 'x', 'y', 'z']
    atom1_line = readlines[atom1_line_num - 1].split()
    atom2_line = readlines[atom2_line_num - 1].split()

    print('fixed  atom: ', atom1_line)
    print('moving atom: ', atom2_line)

    # extracted xyz coordinates of atom 1 and 2:
    r1 = [float(atom1_line[1]), float(atom1_line[2]), float(atom1_line[3])]
    r2 = [float(atom2_line[1]), float(atom2_line[2]), float(atom2_line[3])]

    new_coords = find_point(r1, r2, length)
    new_line = atom2_line
    for i in range(3):
        new_line[i + 1] = str(round(new_coords[i], precision))

    new_line.append('\n')

    readlines[atom2_line_num - 1] = '    '.join(new_line)

    print('new coords : ', new_line)

    with open(filename, 'w') as file:
        file.writelines(readlines)

    with open(filename, 'r') as file:
        readlines = file.readlines()
    
    return readlines

In [ ]:
filename = 'CH3F.xyz'
basis = 'sto-3g'
atom1_line_num = 4
atom2_line_num = 3
precision = 4
run_HF = True
run_CCSD = False
run_FCI = False

run_DFT = False

run_Nbed = False
active_atoms = 2

lengths = np.arange(0.85, 2.35, 0.10)
print(len(lengths), '\n')

hf_energies = np.zeros(len(lengths))
ccsd_energies = np.zeros(len(lengths))
fci_energies = np.zeros(len(lengths))
nbed_energies = np.zeros(len(lengths))
dft_energies = np.zeros(len(lengths))

nbed_config = {
    "geometry" : filename,
    "n_active_atoms" : active_atoms,
    "basis" : basis,
    "xc_functional" : "b3lyp",
    "projector" : "mu",
    "localization" : "spade",
    "convergence" : 1e-6,
    "run_virtual_localization" : True,
    "run_ccsd_emb" : True,
    "run_fci_emb" : False
}

for i in range(len(lengths)):
    print("%.2f" % lengths[i])
    data = edit_xyz(filename, atom1_line_num, atom2_line_num, length = lengths[i], precision = precision)
    print()

    mol = gto.M(atom = filename, basis = basis, spin = 0, charge = 0)

    #HF
    if run_HF:
        mf = scf.RHF(mol).run()
        hf_energies[i] = mf.e_tot
        print('e_nuc = ', mf.energy_nuc())


    #CCSD
    if run_CCSD:
        mf = scf.HF(mol).run()
        mycc = cc.CCSD(mf).run()
        ccsd_energies[i] = mycc.e_tot

    # FCI
    if run_FCI:
        myhf = mol.RHF().run()
        cisolver = fci.FCI(myhf)
        fci_energies[i] = cisolver.kernel()[0]

    # KSDFT
    if run_DFT:
        mf = dft.RKS(mol)
        mf.xc = 'lda,vwn'
        mf = mf.newton()
        dft_energies[i] = mf.kernel()

    # Nbed
    if run_Nbed:
        driver = NbedDriver(**nbed_config)
        driver._run_emb_CCSD(driver.embedded_scf)
        nbed_energies[i] = driver._mu["e_ccsd"]

    print()

# CH3F STO-3G run with FCI: 2m 44.4s

# CH3F 6-31G run without FCI: 10.6s
# CH3F cc-pvdz run without FCI: 34.0s


# mol1 sto-3g HF only: 11m 9.8s

In [ ]:
fig, (ax0, ax1) = plt.subplots(nrows = 1, ncols = 2, sharex = True, sharey = False, figsize = (10,5))

if run_HF:
    ax0.plot(lengths, hf_energies, label = 'HF', marker = '.')
    ax1.plot(lengths, hf_energies - np.min(hf_energies), label = 'HF', marker = '.')
if run_CCSD:
    ax0.plot(lengths, ccsd_energies, label = 'CCSD', marker = '.')
    ax1.plot(lengths, ccsd_energies - np.min(ccsd_energies), label = 'CCSD', marker = '.')
if run_FCI:
    ax0.plot(lengths, fci_energies, label = 'FCI', marker = '.')
    ax1.plot(lengths, fci_energies - np.min(fci_energies), label = 'FCI', marker = '.')
if run_Nbed:
    ax0.plot(lengths, nbed_energies, label = 'Nbed CCSD', marker = '.')
    ax1.plot(lengths, nbed_energies - np.min(nbed_energies), label = 'Nbed CCSD', marker = '.')
if run_DFT:
    ax0.plot(lengths, dft_energies, label = 'KS-DFT', marker = '.')
    ax1.plot(lengths, dft_energies - np.min(dft_energies), label = 'KS-DFT', marker = '.')
ax0.set_ylabel('energy / Hartree')
ax1.set_ylabel('relative energy / Hartree')
fig.suptitle(' '.join([filename[0:-4], ":", basis]))
fig.supxlabel('C-F bond length / angstrom')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
if run_HF:
    plt.plot(lengths, hf_energies - np.min(hf_energies), label = 'HF', marker = '.')
if run_CCSD:
    plt.plot(lengths, ccsd_energies - np.min(ccsd_energies), label = 'CCSD', marker = '.')
if run_FCI:
    plt.plot(lengths, fci_energies - np.min(fci_energies), label = 'FCI', marker = '.')
if run_Nbed:
    plt.plot(lengths, nbed_energies - np.min(nbed_energies), label = 'Nbed', marker = '.')
if run_DFT:
    plt.plot(lengths, dft_energies - np.min(dft_energies), label = 'KS-DFT', marker = '.')
plt.xlabel('C-F bond length / angstrom')
plt.ylabel('energy / Hartree')
plt.title(' '.join([filename[0:-4], ":", basis]))
plt.legend()
plt.show()